### Predict on validation set

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import os
import sys
import torch
import numpy as np
import torch.nn.functional as F
from torch.utils.data import Dataset
from tqdm.notebook import trange, tqdm

In [ ]:
ROOT_DIR = os.path.dirname(os.getcwd())
sys.path.append(ROOT_DIR)

In [ ]:
from src.path import DATA_VALID, MODELS_DIR
from src.metrics import psnr
from models.rednet import RedNet

In [ ]:
class ImgDataset(Dataset):
    def __init__(self, data_dir: str) -> None:
        self.data_dir = data_dir
        self.noisy_imgs ,self.clean_imgs = torch.load(data_dir)

    def __getitem__(self, index):
        x = self.noisy_imgs[index, ...]/255.0
        y = self.clean_imgs[index, ...]/255.0
        return x, y

    def __len__(self):
        return self.noisy_imgs.shape[0]

data = ImgDataset(DATA_VALID)
ValidDataLoader=torch.utils.data.DataLoader(data,batch_size=1,
                                           shuffle=True)

In [ ]:
model = RedNet()
criterion = torch.nn.MSELoss()
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
print(device)
model.eval()
model.to(device)

name = "model-20220420.pt"
if torch.cuda.is_available():
  model.load_state_dict(torch.load(os.path.join(MODELS_DIR, name)))
else:
  model.load_state_dict(torch.load(os.path.join(MODELS_DIR, name), map_location=torch.device('cpu')))

In [ ]:
with tqdm(trainDataLoader, desc=f'Valid', unit='batch', 
                  leave=False) as t1:
            for x_train ,y_train in t1:
                x_train = x_train.to(device)
                y_train = y_train.to(device)

                # Zero the parameter gradients
                optimizer.zero_grad()

                # Forward + backward + optimize
                output = model(x_train)
                loss = criterion(output, y_train)
                loss.backward()
                optimizer.step()
                
                # Append loss
                losses.append(loss)
        
        train_loss.append((sum(losses)/float(len(losses))).item())

        with torch.no_grad():
            losses = []

            with tqdm(testDataLoader, desc=f'Test epoch {epoch}', unit='batch', 
                  leave=False) as t1:
                for x_test, y_test in t1:

                    x_test = x_test.to(device)
                    y_test = y_test.to(device)

                    output = model(x_test)
                    loss = criterion(output, y_test)
                    losses.append(loss)
            
        test_loss.append((sum(losses)/float(len(losses))).item())